In [36]:
import numpy as np 
import pandas as pd
import spacy as sp
import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt

In [37]:
class DenseNormalGamma(tf.keras.layers.Layer):
    
    def __init__(self, units):
        super(DenseNormalGamma, self).__init__()
        self.units = int(units)
        self.dense = tf.keras.layers.Dense(4 * self.units, activation=None)

    def evidence(self, x):
        return tf.nn.softplus(x)

    def call(self, x):
        output = self.dense(x)
        mu, logv, logalpha, logbeta = tf.split(output, 4, axis=-1)
        v = self.evidence(logv)
        alpha = self.evidence(logalpha) + 1
        beta = self.evidence(logbeta)
        return tf.concat([mu, v, alpha, beta], axis=-1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 4 * self.units)
    
    def get_config(self):
        base_config = super(DenseNormalGamma, self).get_config()
        base_config['units'] = self.units
        return base_config

In [38]:
def NIG_NLL(y, gamma, v, alpha, beta, reduce=True):
    twoBlambda = 2*beta*(1+v)

    nll = 0.5*tf.math.log(np.pi/v)  \
        - alpha*tf.math.log(twoBlambda)  \
        + (alpha+0.5) * tf.math.log(v*(y-gamma)**2 + twoBlambda)  \
        + tf.math.lgamma(alpha)  \
        - tf.math.lgamma(alpha+0.5)

    return tf.reduce_mean(nll) if reduce else nll

def NIG_Reg(y, gamma, v, alpha, beta, reduce=True):
    error = tf.abs(y-gamma)

    evi = 2*v+(alpha)
    reg = error*evi

    return tf.reduce_mean(reg) if reduce else reg

def EvidentialRegression(y_true, evidential_output, coeff=1.0):
    
    gamma, v, alpha, beta = tf.split(evidential_output, 4, axis=-1)
    loss_nll = NIG_NLL(y_true, gamma, v, alpha, beta)
    loss_reg = NIG_Reg(y_true, gamma, v, alpha, beta)
    return loss_nll + coeff * loss_reg

In [39]:
# train = pd.read_csv('../input/evidentialdl/df_train.csv')
train = pd.read_csv('../input/evidentialdl/df_test.csv').drop(labels={"molecule_name",'id'},axis=1)

train.sample(5)

,atom_index_0,atom_index_1,type,atom_0,atom_0_x,atom_0_y,atom_0_z,atom_1,atom_1_x,atom_1_y,...,molecule_type_dist_mean_diff,molecule_type_dist_mean_div,molecule_type_dist_max,molecule_type_dist_min,molecule_type_dist_std,molecule_type_dist_std_diff,p_dso,p_pso,p_sd,p_fc
1887234,14,15,6,0,0.5513,-0.0252,-2.137,1,2.44300,-0.8840,...,-0.038060,0.9863,3.130,2.470,0.277000,-2.498,-3.741058,2.921763,0.069877,11.138539
1041597,16,4,0,0,-0.1554,-0.2910,-4.700,0,0.57400,-0.3098,...,0.004383,1.0040,1.100,1.091,0.002821,-1.088,0.671322,0.762751,0.201118,86.649055
2120015,14,6,5,0,-0.2798,-3.4380,1.220,0,1.91600,-2.3180,...,0.367700,1.1390,3.564,2.492,0.320800,-2.332,0.146836,-0.096090,-0.048994,2.489451
1109154,9,1,2,0,0.9014,2.0720,0.805,0,-0.07056,0.2338,...,0.099240,1.0460,2.355,2.152,0.089500,-2.074,0.113766,-0.055186,0.077120,-4.330440
2225192,22,2,2,0,-1.1970,-2.5590,1.025,0,-1.43100,-0.5273,...,-0.095400,0.9580,2.275,2.123,0.035500,-2.238,-0.067584,-0.035072,0.067096,7.096583


In [40]:
train  = pd.DataFrame(train)

In [54]:
list  = ['molecule_x_max','molecule_y_max']
train[:][list]

,molecule_x_max,molecule_y_max
0,1.662,0.000
1,1.662,0.000
2,1.662,0.000
3,1.662,0.000
4,1.662,0.000
...,...,...
2505537,2.516,2.287
2505538,2.516,2.287
2505539,2.516,2.287
2505540,2.516,2.287


'molecule_x_min','molecule_x_var','molecule_x_mean','molecule_x_median','molecule_x_std','molecule_y_min','molecule_y_var','molecule_y_mean','molecule_y_median','molecule_y_std','molecule_z_min','molecule_z_var','molecule_z_mean','molecule_z_median','molecule_z_std']

In [56]:
list=['molecule_x_max','molecule_y_max','molecule_z_max']
Y = train[:]['molecule_volume']
X = train[:][list]
X


,molecule_x_max,molecule_y_max,molecule_z_max
0,1.662,0.000,1.000
1,1.662,0.000,1.000
2,1.662,0.000,1.000
3,1.662,0.000,1.000
4,1.662,0.000,1.000
...,...,...,...
2505537,2.516,2.287,1.459
2505538,2.516,2.287,1.459
2505539,2.516,2.287,1.459
2505540,2.516,2.287,1.459


In [57]:
X = np.asarray(X).astype('float32')

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(3,)),
        tf.keras.layers.Dense(128, activation='relu'), 
        tf.keras.layers.Dropout(0.2),  
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units = 10, activation = 'relu'),
        tf.keras.layers.Dense(units = 5, activation = 'relu'),
        DenseNormalGamma(1)
    ]
)

# compile model
model.compile(
    optimizer = 'adam', 
    loss = EvidentialRegression,
    metrics = ['mse','accuracy']
)

# create early stopping callback
callback1 = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss', 
    mode = 'min',
    patience = 25,
    restore_best_weights = True
)

# create reduce LR callback
callback2 = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss', 
    factor = 0.25, 
    patience = 5, 
    verbose = 0,
    mode = 'min'
)

# fit model to training data
history = model.fit(
    x = X, 
    y = Y, 
    validation_split = 0.25, 
    batch_size = 4000,
    epochs = 200,
    callbacks = [callback1, callback2],
    verbose = 1
)

Epoch 1/200
470/470 [==============================] - 11s 20ms/step - loss: 119.9440 - mse: 10565.2656 - accuracy: 0.0000e+00 - val_loss: 121.5838 - val_mse: 14251.7910 - val_accuracy: 0.0000e+00
Epoch 2/200
470/470 [==============================] - 9s 19ms/step - loss: nan - mse: nan - accuracy: 3.1929e-06 - val_loss: nan - val_mse: nan - val_accuracy: 0.0000e+00
Epoch 3/200
470/470 [==============================] - 10s 20ms/step - loss: nan - mse: nan - accuracy: 4.2572e-06 - val_loss: nan - val_mse: nan - val_accuracy: 0.0000e+00
Epoch 4/200
470/470 [==============================] - 9s 19ms/step - loss: nan - mse: nan - accuracy: 4.2572e-06 - val_loss: nan - val_mse: nan - val_accuracy: 0.0000e+00
Epoch 5/200
470/470 [==============================] - 9s 19ms/step - loss: nan - mse: nan - accuracy: 4.2572e-06 - val_loss: nan - val_mse: nan - val_accuracy: 0.0000e+00
Epoch 6/200
470/470 [==============================] - 10s 21ms/step - loss: nan - mse: nan - accuracy: 4.2572e-06

In [ ]:
y_pred = model.predict(train_vectors)

# compute variance and std from learned parameters
mu, v, alpha, beta = (y_pred[:, i] for i in range(y_pred.shape[1]))

var_a = beta / (alpha - 1)
var_e = beta / (v * (alpha - 1))

In [ ]:
sns.jointplot(x = train['target'], y = mu, kind = 'hex')
plt.xlabel('Target')
plt.ylabel('Predicted Target')
plt.show()

In [ ]:
sns.jointplot(x = np.sqrt(var_e), y = np.sqrt(var_a), kind = 'hex')
plt.xlabel('Epistemic Uncertainty')
plt.ylabel('Aleatoric Uncertainty')
plt.show()

In [ ]:
sns.jointplot(x = train['target'], y = np.sqrt(var_a), kind = 'hex')
plt.xlabel('Target')
plt.ylabel('Aleatoric Uncertainty')
plt.show()

In [ ]:
sns.jointplot(x = mu, y = np.sqrt(var_a), kind = 'hex')
plt.xlabel('Predicted Target')
plt.ylabel('Aleatoric Uncertainty')
plt.show()

In [ ]:
sns.jointplot(x = train['target'], y = np.sqrt(var_e), kind = 'hex')
plt.xlabel('Target')
plt.ylabel('Epistemic Uncertainty')
plt.show()

In [ ]:
sns.jointplot(x = mu, y = np.sqrt(var_e), kind = 'hex')
plt.xlabel('Predicted Target')
plt.ylabel('Epistemic Uncertainty')
plt.show()

In [ ]:
sns.lmplot(
    data = pd.DataFrame({
        'RMSE' : np.sqrt((train['target'] - mu)**2),
        'STD_A' : np.sqrt(var_a)
    }),
    x = 'RMSE',
    y = 'STD_A'
)
plt.xlabel('RMSE')
plt.ylabel('Aleatoric Uncertainty')
plt.show()